# Introduction to XGBoost Spark with GPU

Mortgage is an example of xgboost classifier. In this notebook, we will show you how to load data, train the xgboost model and use this model to predict if someone is "deliquency". Comparing to original XGBoost Spark codes, there're only two API differences.


## Load libraries
First we load some common libraries that both GPU version and CPU version xgboost will use:

In [1]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassifier, XGBoostClassificationModel}
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}

what is new to xgboost-spark users is only `rapids.GpuDataReader`

In [2]:
import ml.dmlc.xgboost4j.scala.spark.rapids.{GpuDataReader, GpuDataset}

Some libraries needed for CPU version are not needed in GPU version any more. The extra libraries needed for CPU are like below:

```scala
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.FloatType
```

## Set your dataset path

In [3]:
// Set the paths of datasets for training and prediction
// You need to update them to your real paths!
val trainPath = "/data/mortgage/csv/train/"
val trainWithEvalPath = "/data/mortgage/csv/trainWithEval/"
val evalPath  = "/data/mortgage/csv/eval/"

trainPath = /data/mortgage/csv/train/
trainWithEvalPath = /data/mortgage/csv/trainWithEval/
evalPath = /data/mortgage/csv/eval/


/data/mortgage/csv/eval/

## Set the schema of the dataset
for mortgage example, the data has 27 columns: 26 features and 1 label. "deinquency_12" is set as label column. The schema will be used to help load data in the future. We also defined some key parameters used in xgboost training process. We also set some basic xgboost parameters here.

In [4]:
val labelColName = "delinquency_12"
val schema = StructType(List(
  StructField("orig_channel", DoubleType),
  StructField("first_home_buyer", DoubleType),
  StructField("loan_purpose", DoubleType),
  StructField("property_type", DoubleType),
  StructField("occupancy_status", DoubleType),
  StructField("property_state", DoubleType),
  StructField("product_type", DoubleType),
  StructField("relocation_mortgage_indicator", DoubleType),
  StructField("seller_name", DoubleType),
  StructField("mod_flag", DoubleType),
  StructField("orig_interest_rate", DoubleType),
  StructField("orig_upb", IntegerType),
  StructField("orig_loan_term", IntegerType),
  StructField("orig_ltv", DoubleType),
  StructField("orig_cltv", DoubleType),
  StructField("num_borrowers", DoubleType),
  StructField("dti", DoubleType),
  StructField("borrower_credit_score", DoubleType),
  StructField("num_units", IntegerType),
  StructField("zip", IntegerType),
  StructField("mortgage_insurance_percent", DoubleType),
  StructField("current_loan_delinquency_status", IntegerType),
  StructField("current_actual_upb", DoubleType),
  StructField("interest_rate", DoubleType),
  StructField("loan_age", DoubleType),
  StructField("msa", DoubleType),
  StructField("non_interest_bearing_upb", DoubleType),
  StructField(labelColName, IntegerType)))

val commParamMap = Map(
  "eta" -> 0.1,
  "gamma" -> 0.1,
  "missing" -> 0.0,
  "max_depth" -> 10,
  "max_leaves" -> 256,
  "grow_policy" -> "depthwise",
  "min_child_weight" -> 30,
  "lambda" -> 1,
  "scale_pos_weight" -> 2,
  "subsample" -> 1,
  "nthread" -> 1,
  "num_round" -> 100)

labelColName = delinquency_12
schema = StructType(StructField(orig_channel,DoubleType,true), StructField(first_home_buyer,DoubleType,true), StructField(loan_purpose,DoubleType,true), StructField(property_type,DoubleType,true), StructField(occupancy_status,DoubleType,true), StructField(property_state,DoubleType,true), StructField(product_type,DoubleType,true), StructField(relocation_mortgage_indicator,DoubleType,true), StructField(seller_name,DoubleType,true), StructField(mod_flag,DoubleType,true), StructField(orig_interest_rate,DoubleType,true), StructField(orig_upb,IntegerType,true), StructField(orig_loan_term,IntegerType,true), StructField(orig_ltv,DoubleType,true), StructField(orig_cltv,DoubleType,true), StructField(num_borrowers,DoubleT...


StructType(StructField(orig_channel,DoubleType,true), StructField(first_home_buyer,DoubleType,true), StructField(loan_purpose,DoubleType,true), StructField(property_type,DoubleType,true), StructField(occupancy_status,DoubleType,true), StructField(property_state,DoubleType,true), StructField(product_type,DoubleType,true), StructField(relocation_mortgage_indicator,DoubleType,true), StructField(seller_name,DoubleType,true), StructField(mod_flag,DoubleType,true), StructField(orig_interest_rate,DoubleType,true), StructField(orig_upb,IntegerType,true), StructField(orig_loan_term,IntegerType,true), StructField(orig_ltv,DoubleType,true), StructField(orig_cltv,DoubleType,true), StructField(num_borrowers,DoubleT...

## Create a new spark session and load data
we must create a new spark session to continue all spark operations. It will also be used to initilize the `GpuDataReader` which is a data reader powered by GPU.

Here's the first API difference, we now use GpuDataReader to load dataset. Similar to original Spark data loading API, GpuDataReader also uses chaining call of "option", "schema","csv". For CPU verions data reader, the code is like below:

```scala
val dataReader = spark.read
```

NOTE: in this notebook, we have uploaded dependency jars when installing toree kernel. If we don't upload them at installation time, we can also upload in notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called after a new spark session is created. We must use it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("mortgage-GPU").getOrCreate
%AddJar file:/data/libs/cudf-0.9.1-cuda10.jar
%AddJar file:/data/libs/xgboost4j_2.11-1.0.0-Beta2.jar
%AddJar file:/data/libs/xgboost4j-spark_2.11-1.0.0-Beta2.jar
// ...
```

In [5]:
val spark = SparkSession.builder().appName("mortgage-gpu").getOrCreate

spark = org.apache.spark.sql.SparkSession@6ec3b17c


org.apache.spark.sql.SparkSession@6ec3b17c

Here's the first API difference, we now use `GpuDataReader` to load dataset. Similar to original Spark data loading API, `GpuDataReader` also uses chaining call of "option", "schema","csv". For `CPU` verion data reader, the code is like below:

```scala
val dataReader = spark.read
```

`featureNames` is used to tell xgboost which columns are `feature` and while column is `label`

In [6]:
val reader = new GpuDataReader(spark).option("header", true).schema(schema)
val featureNames = schema.filter(_.name != labelColName).map(_.name)

reader = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataReader@46845449
featureNames = List(orig_channel, first_home_buyer, loan_purpose, property_type, occupancy_status, property_state, product_type, relocation_mortgage_indicator, seller_name, mod_flag, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, current_loan_delinquency_status, current_actual_upb, interest_rate, loan_age, msa, non_interest_bearing_upb)


List(orig_channel, first_home_buyer, loan_purpose, property_type, occupancy_status, property_state, product_type, relocation_mortgage_indicator, seller_name, mod_flag, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, current_loan_delinquency_status, current_actual_upb, interest_rate, loan_age, msa, non_interest_bearing_upb)

Now we can use `dataReader` to read data directly. However, in CPU version, we have to use `VectorAssembler` to assemble all feature columns into one column. The reason will be explained later. the CPU version code is as below:

```scala
object Vectorize {
  def apply(df: DataFrame, featureNames: Seq[String], labelName: String): DataFrame = {
    val toFloat = df.schema.map(f => col(f.name).cast(FloatType))
    new VectorAssembler()
      .setInputCols(featureNames.toArray)
      .setOutputCol("features")
      .transform(df.select(toFloat:_*))
      .select(col("features"), col(labelName))
  }
}

val trainSet = reader.csv(trainPath)
val evalSet = reader.csv(evalPath)
trainSet = Vectorize(trainSet, featureCols, labelName)
evalSet = Vectorize(evalSet, featureCols, labelName)

```

While with GpuDataReader, `VectorAssembler` is not needed any more. We can simply read data by:

In [7]:
val trainSet = reader.csv(trainPath)
val trainWithEvalSet = reader.csv(trainWithEvalPath)
val evalSet = reader.csv(evalPath)

trainSet = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@3ca0708c
trainWithEvalSet = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@234184fd
evalSet = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@2dbe1638


ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@2dbe1638

## Add XGBoost parameters for GPU version

Another modification is `num_workers` should be set to the number of machines with GPU in Spark cluster, while it can be set to the number of your CPU cores in CPU version. CPU version parameters:

```scala
// difference in parameters
"tree_method" -> "hist", 
"num_workers" -> 12
```

In [8]:
val xgbParamFinal = commParamMap ++ Map("tree_method" -> "gpu_hist", "num_workers" -> 1)

xgbParamFinal = Map(min_child_weight -> 30, grow_policy -> depthwise, scale_pos_weight -> 2, num_workers -> 1, subsample -> 1, lambda -> 1, max_depth -> 10, num_round -> 100, missing -> 0.0, tree_method -> gpu_hist, eta -> 0.1, max_leaves -> 256, gamma -> 0.1, nthread -> 1)


Map(min_child_weight -> 30, grow_policy -> depthwise, scale_pos_weight -> 2, num_workers -> 1, subsample -> 1, lambda -> 1, max_depth -> 10, num_round -> 100, missing -> 0.0, tree_method -> gpu_hist, eta -> 0.1, max_leaves -> 256, gamma -> 0.1, nthread -> 1)

In [9]:
val xgbClassifier = new XGBoostClassifier(xgbParamFinal)
      .setLabelCol(labelColName)
      // === diff ===
      .setFeaturesCols(featureNames)

xgbClassifier = xgbc_12a415c93943


xgbc_12a415c93943

## Benchmark and train
The benchmark object is for calculating training time. We will use it to compare with xgboost in CPU version.

We also support training with evaluation sets in 2 ways as same as CPU version behavior:

* API `setEvalSets` after initializing an XGBoostClassifier

```scala
xgbClassifier.setEvalSets(Map("eval" -> evalSet))

```

* parameter `eval_sets` when initializing an XGBoostClassifier

```scala
val paramMapWithEval = paramMap + ("eval_sets" -> Map("eval" -> evalSet))
val xgbClassifierWithEval = new XGBoostClassifier(paramMapWithEval)
```

in this notebook, we use API method to set evaluation sets.

In [10]:
xgbClassifier.setEvalSets(Map("eval" -> trainWithEvalSet))

xgbc_12a415c93943

In [11]:
object Benchmark {
  def time[R](phase: String)(block: => R): (R, Float) = {
    val t0 = System.currentTimeMillis
    val result = block // call-by-name
    val t1 = System.currentTimeMillis
    println("Elapsed time [" + phase + "]: " + ((t1 - t0).toFloat / 1000) + "s")
    (result, (t1 - t0).toFloat / 1000)
  }
}

// Start training
println("\n------ Training ------")
val (xgbClassificationModel, _) = Benchmark.time("train") {
  xgbClassifier.fit(trainSet)
}



------ Training ------
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.124, DMLC_TRACKER_PORT=9092, DMLC_NUM_WORKER=1}
Elapsed time [train]: 14.013s


defined object Benchmark
xgbClassificationModel = xgbc_12a415c93943


xgbc_12a415c93943

## Transformation and evaluation
We use `evalSet` to evaluate our model and use some key columns to show our predictions. Finally we use `MulticlassClassificationEvaluator` to calculate an overall accuracy of our predictions.

In [12]:
println("\n------ Transforming ------")
val (results, _) = Benchmark.time("transform") {
  val ret = xgbClassificationModel.transform(evalSet).cache()
  ret.foreachPartition(_ => ())
  ret
}
results.select("orig_channel", labelColName,"rawPrediction","probability","prediction").show(10)

println("\n------Accuracy of Evaluation------")
val evaluator = new MulticlassClassificationEvaluator().setLabelCol(labelColName)
val accuracy = evaluator.evaluate(results)
println(accuracy)


------ Transforming ------
Elapsed time [transform]: 5.483s
+------------+--------------+--------------------+--------------------+----------+
|orig_channel|delinquency_12|       rawPrediction|         probability|prediction|
+------------+--------------+--------------------+--------------------+----------+
|         0.0|           0.0|[-1.3482570648193...|[0.99986517429351...|       0.0|
|         0.0|           0.0|[3.86238098144531...|[1.00003862380981...|       0.0|
|         0.0|           0.0|[9.65595245361328...|[1.00000965595245...|       0.0|
|         0.0|           0.0|[-1.1876225471496...|[0.99988123774528...|       0.0|
|         0.0|           0.0|[-7.2091817855834...|[0.99992790818214...|       0.0|
|         0.0|           0.0|[4.38451766967773...|[1.00043845176696...|       0.0|
|         0.0|           0.0|[7.64369964599609...|[1.00076436996459...|       0.0|
|         0.0|           0.0|[4.24027442932128...|[1.00042402744293...|       0.0|
|         0.0|           0

results = [orig_channel: float, first_home_buyer: float ... 29 more fields]
evaluator = mcEval_037d0924816e
accuracy = 0.9990008308279212


0.9990008308279212

## Save the model to disk and load model
We save the model to disk and then load it to memory. We can use the loaded model to do a new prediction.

In [13]:
xgbClassificationModel.write.overwrite.save("/data/model/mortgage")

val modelFromDisk = XGBoostClassificationModel.load("/data/model/mortgage")

val (results2, _) = Benchmark.time("transform2") {
  modelFromDisk.transform(evalSet)
}
results2.show(10)

Elapsed time [transform2]: 0.04s
+------------+----------------+------------+-------------+----------------+--------------+------------+-----------------------------+-----------+--------+------------------+--------+--------------+--------+---------+-------------+----+---------------------+---------+-----+--------------------------+-------------------------------+------------------+-------------+--------+-------+------------------------+--------------+--------------------+--------------------+----------+
|orig_channel|first_home_buyer|loan_purpose|property_type|occupancy_status|property_state|product_type|relocation_mortgage_indicator|seller_name|mod_flag|orig_interest_rate|orig_upb|orig_loan_term|orig_ltv|orig_cltv|num_borrowers| dti|borrower_credit_score|num_units|  zip|mortgage_insurance_percent|current_loan_delinquency_status|current_actual_upb|interest_rate|loan_age|    msa|non_interest_bearing_upb|delinquency_12|       rawPrediction|         probability|prediction|
+------------+-

modelFromDisk = xgbc_12a415c93943
results2 = [orig_channel: float, first_home_buyer: float ... 29 more fields]


[orig_channel: float, first_home_buyer: float ... 29 more fields]

In [14]:
spark.close()